#ProteinMPNN
This notebook is intended as a quick demo, more features to come!

이 노트북은 ProteinMPNN의 저자들이 만든 colab notebook에 기반하여 약간 수정하였습니다.

https://colab.research.google.com/github/dauparas/ProteinMPNN/blob/main/colab_notebooks/quickdemo.ipynb


초안은 서울대학교 약학대학 이주용 교수님 연구실에서 작성하였으며, 실습목적에 맞게 편집되었습니다.

## Clone github repository
----

우선 github에 존재하는 ProteinMPNN의 소스코드를 colab으로 다운 받습니다.

In [1]:
#@title Clone github repo
import json, time, os, sys, glob

if not os.path.isdir("ProteinMPNN"):
  os.system("git clone -q https://github.com/dauparas/ProteinMPNN.git")
sys.path.append('/content/ProteinMPNN')

## Model을 colab에 setup 하기
-----

본 단계에서는 다운받은 github 코드를 colab 환경에 설치해줍니다.

여기서는 ProteinMPNN의 model version을 선택할 수 있습니다.

모델의 version은 v_48_002, v_48_010, v_48_020, v_48_030 등을 선택할 수 있습니다.

각 버젼은 ProteinMPNN을 학습시킬 때, 단백질의 backbone에 어느 정도의 noise를 주었느냐에 따라서 달라지게 됩니다.

예를 들어 여기에서 v_48_020 버젼은 backbone 원자들에게 0.2 angstrom의 noise를 준 데이터를 이용해서 학습시킨 모델 파라미터 입니다.

따라서, 각 모델에 따라서 디자인되는 서열의 종류가 달라질 것이기 때문에 문제에 따라서 다양한 버젼을 가지고 각 결과가 어떻게 달라질지 확인해보는 것도 흥미로울 것입니다.



In [ ]:
#@title Setup Model
import matplotlib.pyplot as plt
import shutil
import warnings
import numpy as np
import torch
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split, Subset
import copy
import torch.nn as nn
import torch.nn.functional as F
import random
import os.path
from protein_mpnn_utils import loss_nll, loss_smoothed, gather_edges, gather_nodes, gather_nodes_t, cat_neighbors_nodes, _scores, _S_to_seq, tied_featurize, parse_PDB
from protein_mpnn_utils import StructureDataset, StructureDatasetPDB, ProteinMPNN

device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
#v_48_010=version with 48 edges 0.10A noise
model_name = "v_48_020" #@param ["v_48_002", "v_48_010", "v_48_020", "v_48_030"]


backbone_noise=0.00               # Standard deviation of Gaussian noise to add to backbone atoms

path_to_model_weights='/content/ProteinMPNN/vanilla_model_weights'
hidden_dim = 128
num_layers = 3
model_folder_path = path_to_model_weights
if model_folder_path[-1] != '/':
    model_folder_path = model_folder_path + '/'
checkpoint_path = model_folder_path + f'{model_name}.pt'

checkpoint = torch.load(checkpoint_path, map_location=device)
print('Number of edges:', checkpoint['num_edges'])
noise_level_print = checkpoint['noise_level']
print(f'Training noise level: {noise_level_print}A')
model = ProteinMPNN(num_letters=21, node_features=hidden_dim, edge_features=hidden_dim, hidden_dim=hidden_dim, num_encoder_layers=num_layers, num_decoder_layers=num_layers, augment_eps=backbone_noise, k_neighbors=checkpoint['num_edges'])
model.to(device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()
print("Model loaded")

## Tied residue pairs
------

ProteinMPNN의 경우 homo-oligomer를 모델링 할 때는 symmetry를 고려하여 동일한 위치에
동일한 아미노산이 위치하도록 constraint를 줄 수 있습니다.

이를 tied residue pair라고 표현하고 이를 고려하기 위한 함수를 이 단계에서 정의합니다.

In [3]:
#@title Helper functions
def make_tied_positions_for_homomers(pdb_dict_list):
    my_dict = {}
    for result in pdb_dict_list:
        all_chain_list = sorted([item[-1:] for item in list(result) if item[:9]=='seq_chain']) #A, B, C, ...
        tied_positions_list = []
        chain_length = len(result[f"seq_chain_{all_chain_list[0]}"])
        for i in range(1,chain_length+1):
            temp_dict = {}
            for j, chain in enumerate(all_chain_list):
                temp_dict[chain] = [i] #needs to be a list
            tied_positions_list.append(temp_dict)
        my_dict[result['name']] = tied_positions_list
    return my_dict

## 서열 디자인 예시
------

본 예제에서는 GLP-1/GLP-1 receptor complex를 기반으로 하여 새로운 GLP-1 receptor에 결합할 수 있는 peptide sequence를 만들어 보겠습니다.

Target PDB ID는 6X18입니다.
https://www.rcsb.org/structure/6x18

이 crystal외에도 더 많은 GLP-1 receptor 구조들이 존재합니다.

## Input options
-----


`pdb`: pdb id를 입력해줍니다. 만일 자신의 **pdb 파일을 업로드**하고자 한다면 **blank**로 남겨둡니다.

`homomer`: homo-oligomer인지 여부. 일단 GLP-1R의 경우, homo-oligomer가 아니기 때문에 체크 박스를 빈 칸으로 남겨두겠습니다.

`designed_chain`: 서열을 새롭게 디자인하고자 하는 체인의 ID를 넣어줍니다. 6X18의 경우, GLP-1에 해당하는 chain ID는 P입니다.

`fixed_chain`: 아미노산을 고정시킬 chain의 ID를 입력합니다. 서로 다른 chain들은 comma로 나누어 줍니다.


## Design options
-------
`num_seqs`: 디자인해서 생성할 서열의 개수를 의미합니다.
ProteinMPNN에서는 각 실행이 독립적이고 확률적으로 서열을 생성하므로 여러번 돌릴 때마다 대체로 다른 서열이 나오게 됩니다.
그러나 문제에 따라서 같은 서열이 반복적으로 나올 수 있습니다.
따라서 가급적 여러번 돌리면 돌릴 수록 좋은 서열을 얻을 확률이 높아집니다.


`sampling_temp`: 서열 샘플링 시 사용하는 temperature입니다.
ProteinMPNN에서는 새로운 서열 생성시 각 step에서 20개의 아미노산 + 1개의 X (unknown) 아미노산 중에 하나를 골라서 반복적으로 서열을 생성하게 됩니다.

각 step에서 21개의 AA의 점수를 계산해서 이 21개의 점수 값을 바탕으로 다음에 올 아미노산을 선택하게 됩니다.

이 때 각 AA의 확률을 인공지능이 매기는 점수로 부터 통계열역학의 볼츠만 factor 식을 이용해서 결정하게 됩니다.

$$
p_i(x)=\frac{\exp(-z_{i}(x)/kT)}{\sum_{x=1}^{21} \exp(-z_{i}(x)/kT)}
$$

위 식에서 $z_i(x)$는 $i$ 번째 위치에 $x$라는 AA가 올 점수이고, $k$는 상수, $T$는 온도입니다.

따라서, $T=0$이면 각 $i$번째 step에서 항상 점수가 가장 높은, 가장 확률이 높은 AA가 오게 됩니다.

반대로 $T>>0$ 이면, 점수와 크게 상관없이 모든 AA들이 비슷한 확률을 가지고 생성될 수 있습니다.

따라서 $T$는 서열 생성의 다양성을 조절하는 파라미터가 됩니다.

기본 파라미터는 $T=0.1$입니다.

실제 단백질 디자인 시에는 서로 다른 $T$를 가지고 여러 번 수행을 해보고 score가 어떻게 변하는지 얼마나 다양한 서열이 나오는지를 분석해보는 것을 추천 드립니다.



In [ ]:
import re
from google.colab import files
import numpy as np

#########################
def get_pdb(pdb_code=""):
  if pdb_code is None or pdb_code == "":
    upload_dict = files.upload()
    pdb_string = upload_dict[list(upload_dict.keys())[0]]
    with open("tmp.pdb","wb") as out: out.write(pdb_string)
    return "tmp.pdb"
  else:
    os.system(f"wget -qnc https://files.rcsb.org/view/{pdb_code}.pdb")
    return f"{pdb_code}.pdb"

#@markdown ### Input Options
pdb='6X18' #@param {type:"string"}
pdb_path = get_pdb(pdb)
#@markdown - pdb code (leave blank to get an upload prompt)

homomer = False #@param {type:"boolean"}
designed_chain = "P" #@param {type:"string"}
fixed_chain = "A,B,G,N,R" #@param {type:"string"}

if designed_chain == "":
  designed_chain_list = []
else:
  designed_chain_list = re.sub("[^A-Za-z]+",",", designed_chain).split(",")

if fixed_chain == "":
  fixed_chain_list = []
else:
  fixed_chain_list = re.sub("[^A-Za-z]+",",", fixed_chain).split(",")

chain_list = list(set(designed_chain_list + fixed_chain_list))

#@markdown - specified which chain(s) to design and which chain(s) to keep fixed.
#@markdown   Use comma:`A,B` to specifiy more than one chain

#chain = "A" #@param {type:"string"}
#pdb_path_chains = chain
##@markdown - Define which chain to redesign

#@markdown ### Design Options
num_seqs = 8 # @param ["1", "2", "4", "8", "16", "32", "64"] {type:"raw"}
num_seq_per_target = num_seqs

#@markdown - Sampling temperature for amino acids, T=0.0 means taking argmax, T>>1.0 means sample randomly.
sampling_temp = "0.1" #@param ["0.0001", "0.1", "0.15", "0.2", "0.25", "0.3", "0.5"]



save_score=1                      # 0 for False, 1 for True; save score=-log_prob to npy files
save_probs=1                      # 0 for False, 1 for True; save MPNN predicted probabilites per position
score_only=1                      # 0 for False, 1 for True; score input backbone-sequence pairs
conditional_probs_only=1          # 0 for False, 1 for True; output conditional probabilities p(s_i given the rest of the sequence and backbone)
conditional_probs_only_backbone=1 # 0 for False, 1 for True; if true output conditional probabilities p(s_i given backbone)

batch_size=1                      # Batch size; can set higher for titan, quadro GPUs, reduce this if running out of GPU memory
max_length=20000                  # Max sequence length

out_folder='.'                    # Path to a folder to output sequences, e.g. /home/out/
jsonl_path=''                     # Path to a folder with parsed pdb into jsonl
omit_AAs='X'                      # Specify which amino acids should be omitted in the generated sequence, e.g. 'AC' would omit alanine and cystine.

pssm_multi=0.0                    # A value between [0.0, 1.0], 0.0 means do not use pssm, 1.0 ignore MPNN predictions
pssm_threshold=0.0                # A value between -inf + inf to restric per position AAs
pssm_log_odds_flag=0               # 0 for False, 1 for True
pssm_bias_flag=0                   # 0 for False, 1 for True


##############################################################

folder_for_outputs = out_folder

NUM_BATCHES = num_seq_per_target//batch_size
BATCH_COPIES = batch_size
temperatures = [float(item) for item in sampling_temp.split()]
omit_AAs_list = omit_AAs
alphabet = 'ACDEFGHIKLMNPQRSTVWYX'

omit_AAs_np = np.array([AA in omit_AAs_list for AA in alphabet]).astype(np.float32)

chain_id_dict = None
fixed_positions_dict = None
pssm_dict = None
omit_AA_dict = None
bias_AA_dict = None
tied_positions_dict = None
bias_by_res_dict = None
bias_AAs_np = np.zeros(len(alphabet))


###############################################################
pdb_dict_list = parse_PDB(pdb_path, input_chain_list=chain_list)
dataset_valid = StructureDatasetPDB(pdb_dict_list, truncate=None, max_length=max_length)

chain_id_dict = {}
chain_id_dict[pdb_dict_list[0]['name']]= (designed_chain_list, fixed_chain_list)

print(chain_id_dict)
for chain in chain_list:
  l = len(pdb_dict_list[0][f"seq_chain_{chain}"])
  print(f"Length of chain {chain} is {l}")

if homomer:
  tied_positions_dict = make_tied_positions_for_homomers(pdb_dict_list)
else:
  tied_positions_dict = None

In [ ]:
#@title RUN
with torch.no_grad():
  print('Generating sequences...')
  for ix, protein in enumerate(dataset_valid):
    score_list = []
    all_probs_list = []
    all_log_probs_list = []
    S_sample_list = []
    seq_list = []
    batch_clones = [copy.deepcopy(protein) for i in range(BATCH_COPIES)]
    X, S, mask, lengths, chain_M, chain_encoding_all, chain_list_list, visible_list_list, masked_list_list, masked_chain_length_list_list, chain_M_pos, omit_AA_mask, residue_idx, dihedral_mask, tied_pos_list_of_lists_list, pssm_coef, pssm_bias, pssm_log_odds_all, bias_by_res_all, tied_beta = tied_featurize(batch_clones, device, chain_id_dict, fixed_positions_dict, omit_AA_dict, tied_positions_dict, pssm_dict, bias_by_res_dict)
    pssm_log_odds_mask = (pssm_log_odds_all > pssm_threshold).float() #1.0 for true, 0.0 for false
    name_ = batch_clones[0]['name']

    randn_1 = torch.randn(chain_M.shape, device=X.device)
    log_probs = model(X, S, mask, chain_M*chain_M_pos, residue_idx, chain_encoding_all, randn_1)
    mask_for_loss = mask*chain_M*chain_M_pos
    scores = _scores(S, log_probs, mask_for_loss)
    native_score = scores.cpu().data.numpy()

    for temp in temperatures:
        for j in range(NUM_BATCHES):
            randn_2 = torch.randn(chain_M.shape, device=X.device)
            if tied_positions_dict == None:
                sample_dict = model.sample(X, randn_2, S, chain_M, chain_encoding_all, residue_idx, mask=mask, temperature=temp, omit_AAs_np=omit_AAs_np, bias_AAs_np=bias_AAs_np, chain_M_pos=chain_M_pos, omit_AA_mask=omit_AA_mask, pssm_coef=pssm_coef, pssm_bias=pssm_bias, pssm_multi=pssm_multi, pssm_log_odds_flag=bool(pssm_log_odds_flag), pssm_log_odds_mask=pssm_log_odds_mask, pssm_bias_flag=bool(pssm_bias_flag), bias_by_res=bias_by_res_all)
                S_sample = sample_dict["S"]
            else:
                sample_dict = model.tied_sample(X, randn_2, S, chain_M, chain_encoding_all, residue_idx, mask=mask, temperature=temp, omit_AAs_np=omit_AAs_np, bias_AAs_np=bias_AAs_np, chain_M_pos=chain_M_pos, omit_AA_mask=omit_AA_mask, pssm_coef=pssm_coef, pssm_bias=pssm_bias, pssm_multi=pssm_multi, pssm_log_odds_flag=bool(pssm_log_odds_flag), pssm_log_odds_mask=pssm_log_odds_mask, pssm_bias_flag=bool(pssm_bias_flag), tied_pos=tied_pos_list_of_lists_list[0], tied_beta=tied_beta, bias_by_res=bias_by_res_all)
            # Compute scores
                S_sample = sample_dict["S"]
            log_probs = model(X, S_sample, mask, chain_M*chain_M_pos, residue_idx, chain_encoding_all, randn_2, use_input_decoding_order=True, decoding_order=sample_dict["decoding_order"])
            mask_for_loss = mask*chain_M*chain_M_pos
            scores = _scores(S_sample, log_probs, mask_for_loss)
            scores = scores.cpu().data.numpy()
            all_probs_list.append(sample_dict["probs"].cpu().data.numpy())
            all_log_probs_list.append(log_probs.cpu().data.numpy())
            S_sample_list.append(S_sample.cpu().data.numpy())
            for b_ix in range(BATCH_COPIES):
                masked_chain_length_list = masked_chain_length_list_list[b_ix]
                masked_list = masked_list_list[b_ix]
                seq_recovery_rate = torch.sum(torch.sum(torch.nn.functional.one_hot(S[b_ix], 21)*torch.nn.functional.one_hot(S_sample[b_ix], 21),axis=-1)*mask_for_loss[b_ix])/torch.sum(mask_for_loss[b_ix])
                seq = _S_to_seq(S_sample[b_ix], chain_M[b_ix])
                score = scores[b_ix]
                score_list.append(score)
                native_seq = _S_to_seq(S[b_ix], chain_M[b_ix])
                if b_ix == 0 and j==0 and temp==temperatures[0]:
                    start = 0
                    end = 0
                    list_of_AAs = []
                    for mask_l in masked_chain_length_list:
                        end += mask_l
                        list_of_AAs.append(native_seq[start:end])
                        start = end
                    native_seq = "".join(list(np.array(list_of_AAs)[np.argsort(masked_list)]))
                    l0 = 0
                    for mc_length in list(np.array(masked_chain_length_list)[np.argsort(masked_list)])[:-1]:
                        l0 += mc_length
                        native_seq = native_seq[:l0] + '/' + native_seq[l0:]
                        l0 += 1
                    sorted_masked_chain_letters = np.argsort(masked_list_list[0])
                    print_masked_chains = [masked_list_list[0][i] for i in sorted_masked_chain_letters]
                    sorted_visible_chain_letters = np.argsort(visible_list_list[0])
                    print_visible_chains = [visible_list_list[0][i] for i in sorted_visible_chain_letters]
                    native_score_print = np.format_float_positional(np.float32(native_score.mean()), unique=False, precision=4)
                    line = '>{}, score={}, fixed_chains={}, designed_chains={}, model_name={}\n{}\n'.format(name_, native_score_print, print_visible_chains, print_masked_chains, model_name, native_seq)
                    print(line.rstrip())
                start = 0
                end = 0
                list_of_AAs = []
                for mask_l in masked_chain_length_list:
                    end += mask_l
                    list_of_AAs.append(seq[start:end])
                    start = end

                seq = "".join(list(np.array(list_of_AAs)[np.argsort(masked_list)]))
                seq_list.append(seq)
                l0 = 0
                for mc_length in list(np.array(masked_chain_length_list)[np.argsort(masked_list)])[:-1]:
                    l0 += mc_length
                    seq = seq[:l0] + '/' + seq[l0:]
                    l0 += 1
                score_print = np.format_float_positional(np.float32(score), unique=False, precision=4)
                seq_rec_print = np.format_float_positional(np.float32(seq_recovery_rate.detach().cpu().numpy()), unique=False, precision=4)
                line = '>T={}, sample={}, score={}, seq_recovery={}\n{}\n'.format(temp,b_ix,score_print,seq_rec_print,seq)
                print(line.rstrip())


all_probs_concat = np.concatenate(all_probs_list)
all_log_probs_concat = np.concatenate(all_log_probs_list)
S_sample_concat = np.concatenate(S_sample_list)

## Analyzing results

------
colab 버젼의 ProteinMPNN에서는 위와 같이 간단한 output이 나오게 됩니다.

각 디자인 된 sequence에서 각 서열의 ProteinMPNN이 측정한 score가 나오게 됩니다.

score는 sum of log probability에 해당합니다.

$$
S = \sum_{i} -\log (p_i)
$$

따라서 **값이 작으면 작을수록** (ProteinMPNN이 추측하기에) **더 probable한 sequence**라는 뜻입니다.



## Things to consider
------
1. WT보다 좋은 sequence가 얼마나 생성되었는지?

2. 각 위치에서 아미노산의 선호도는 어떻게 되는지?

3. 각 위치에서 선호되는 아미노산은 왜 선호되는 걸까?

In [ ]:
#@markdown ### Sampling temperature adjusted amino acid probabilties
import plotly.express as px
fig = px.imshow(all_probs_concat.mean(0)[:30,:].T,
                labels=dict(x="positions", y="amino acids", color="probability"),
                y=list(alphabet),
                template="simple_white"
               )

fig.update_xaxes(side="top")


fig.show()